# Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier
 
from sklearn.linear_model import LinearRegression
 
import numpy as np
from sklearn.model_selection import KFold
 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
 
import os
import math
import copy

In [ ]:
# Mouting with google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Modern Problems in Technology'

 check				    no2_stations.csv	   test.csv
 cluster_center_nonscale_data.csv  'Nội dung Slide.gdoc'   Timeline.gsheet
 data_processed_withoutscale.csv    Pipline.gdoc
 data_processed_withscale.csv	    Report.gdoc


## Config path and variable

In [ ]:
data_folder = '/content/drive/MyDrive/Modern Problems in Technology'

# Choose one of 2 versions below:
# Uncomment below line to choose scaled data version
# data_version = 'data_processed_withscale.csv'

# Uncomment below line to choose non-scaled data version
data_version = 'data_processed_withoutscale.csv'


data = pd.read_csv(os.path.join(data_folder, data_version))

data


,lat,lon,NO2,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,pplt_dens,road_dens,cluster
0,21.0491,105.8831,0.000000,391.864014,102794.0,66.589302,11.087100,4.69022,5.980500,5.019875,4.711437,4.758125,27.722601,1828.880005,4
1,21.0491,105.8831,0.000000,558.942993,102699.0,65.773903,12.090000,3.59040,2.657062,4.092187,8.700875,7.603187,27.722601,1828.880005,4
2,21.0491,105.8831,41.310400,326.197998,102453.0,69.259697,13.018900,2.96978,5.755687,6.602938,8.837000,9.167312,27.722601,1828.880005,4
3,21.0491,105.8831,39.379204,197.524994,102181.0,78.663696,15.516100,2.77573,8.956062,10.830875,13.301438,13.640625,27.722601,1828.880005,4
4,21.0491,105.8831,39.901879,287.825012,102072.0,75.866699,17.658199,3.12869,12.547000,12.891687,15.638813,12.034750,27.722601,1828.880005,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,21.0197,105.8147,18.995833,454.903015,102068.0,43.484299,16.768499,3.66034,7.437375,5.100375,3.930875,6.508750,442.738007,4993.080078,5
5456,21.0197,105.8147,23.241667,400.079010,101755.0,49.469700,19.499901,2.41066,13.452063,12.091188,5.180625,9.206437,442.738007,4993.080078,5
5457,21.0197,105.8147,16.816667,326.407013,101510.0,66.391800,22.555500,1.45771,17.722375,16.551500,13.518500,18.223813,442.738007,4993.080078,5
5458,21.0197,105.8147,21.595833,468.182007,101796.0,64.951401,23.062800,4.10253,16.676687,14.273500,16.221062,18.177937,442.738007,4993.080078,5


## Reading data version

In [ ]:
data = pd.read_csv(os.path.join(data_folder, data_version))

data


,lat,lon,NO2,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,pplt_dens,road_dens,cluster
0,21.0491,105.8831,0.000000,391.864014,102794.0,66.589302,11.087100,4.69022,5.980500,5.019875,4.711437,4.758125,27.722601,1828.880005,4
1,21.0491,105.8831,0.000000,558.942993,102699.0,65.773903,12.090000,3.59040,2.657062,4.092187,8.700875,7.603187,27.722601,1828.880005,4
2,21.0491,105.8831,41.310400,326.197998,102453.0,69.259697,13.018900,2.96978,5.755687,6.602938,8.837000,9.167312,27.722601,1828.880005,4
3,21.0491,105.8831,39.379204,197.524994,102181.0,78.663696,15.516100,2.77573,8.956062,10.830875,13.301438,13.640625,27.722601,1828.880005,4
4,21.0491,105.8831,39.901879,287.825012,102072.0,75.866699,17.658199,3.12869,12.547000,12.891687,15.638813,12.034750,27.722601,1828.880005,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,21.0197,105.8147,18.995833,454.903015,102068.0,43.484299,16.768499,3.66034,7.437375,5.100375,3.930875,6.508750,442.738007,4993.080078,5
5456,21.0197,105.8147,23.241667,400.079010,101755.0,49.469700,19.499901,2.41066,13.452063,12.091188,5.180625,9.206437,442.738007,4993.080078,5
5457,21.0197,105.8147,16.816667,326.407013,101510.0,66.391800,22.555500,1.45771,17.722375,16.551500,13.518500,18.223813,442.738007,4993.080078,5
5458,21.0197,105.8147,21.595833,468.182007,101796.0,64.951401,23.062800,4.10253,16.676687,14.273500,16.221062,18.177937,442.738007,4993.080078,5


In [ ]:
# data = data[data.cluster != 2]
# Unique cluster value
data['cluster'].unique()

array([4, 1, 2, 3, 0, 5])

## Remove Invalid NO2 values

In [ ]:
# Run this if you want to remove invalid no2 values

data = data[(data.NO2 > 0) & (data.NO2 < 100)]
data[data.NO2 > 100]

,lat,lon,NO2,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,pplt_dens,road_dens,cluster


## Drop some columns of data



In [ ]:
def drop_cols(df, colName):
    if colName in df:
      df.drop(colName, axis='columns', inplace=True)

# drop_cols(data, "press")

## Create multiple datatset by dropping each colums of original dataset


In [ ]:
# columns = data.columns
# for col in range(3, 15):
#   print(columns[col])
# dropDatas = [data]
dropDatas = []
# print(dropDatas[0])
# for i in range(3, 15):
#   print(columns[i])
#   data_copy = copy.deepcopy(data)
#   drop_cols(data_copy, columns[i])
#   dropDatas.append(data_copy)
  # print(len(dropDatas[i-3].columns))
data_copy = copy.deepcopy(data)
drop_cols(data_copy, "cluster")
dropDatas.append(data_copy)
len(dropDatas)

1

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


## Convert to numpy for feeding model


In [ ]:
for idx in range(2, 3):
  print("---------------------------------------------")
  if (idx == 0):
      print()
      print("Original data")
      print()
      print("---------------------------------------------")
  else:
      print()
      print("Data without cluster")
      print()
      print("---------------------------------------------")
  
  tmpData = dropDatas[idx-2]
  X = tmpData.drop('NO2', axis = 'columns')
  X_arr = X.to_numpy()

  y = tmpData['NO2']
  y_arr = y.to_numpy()

  # print(X.shape, y.shape)


  # # List of RMSE on all folds
  # rmses_rf = []
  # mapes_rf = []
  # r2s_rf = []

  # rmses_lr = []
  # mapes_lr = []
  # r2s_lr = []

  # # Fold tracking
  # i = 1
  # # Depth of Random Forest
  # depth = 50
  # # Number of split
  # split_number = 5

  # kf = KFold(n_splits=split_number, random_state=None, shuffle=True)
  # kf.get_n_splits(X)

  # for train_index, test_index in kf.split(X):
  #   min_rmse_rf = 10000
  #   min_r2_rf = 10000
  #   min_mape_rf = 10000

  #   min_rmse_lr = 10000
  #   min_r2_lr = 10000
  #   min_mape_lr = 10000

  #   # print("TRAIN:", train_index, "TEST:", test_index)
  #   X_train, X_test = X_arr[train_index], X_arr[test_index]
  #   y_train, y_test = y_arr[train_index], y_arr[test_index]
  #   print(len(train_index), len(test_index))

  #   print('Fold {}:'.format(i))

  #   # Random Forest
  #   for d in range(1, depth):
  #     rf = RandomForestRegressor(n_estimators = d, random_state = 20)
  #     rf.fit(X_train, y_train)
  #     rf_predicts = rf.predict(X_test)
  #     # Calculate RMSE
  #     rmse_rf = math.sqrt(mean_squared_error(y_test, rf_predicts))
  #     # Calculate R2 Score
  #     r2_rf = r2_score(y_test, rf_predicts)
  #     # Calculate mean absolute error (MAE NOT MAPE)
  #     # mape_rf = mean_absolute_error(y_test, rf_predicts)
  #     mape_rf = mean_absolute_percentage_error(y_test, rf_predicts)
      
  #     # Saving best errors
  #     if rmse_rf < min_rmse_rf:
  #       min_rmse_rf = rmse_rf
  #     if r2_rf < min_r2_rf:
  #       min_r2_rf = r2_rf
  #     if mape_rf < min_mape_rf:
  #       min_mape_rf = mape_rf

  #   rmses_rf.append(min_rmse_rf)
  #   r2s_rf.append(min_r2_rf)
  #   mapes_rf.append(min_mape_rf)

  #   print('Min RMSE for Random Forest:'.format(i), min_rmse_rf)
  #   print('Min R2 for Random Forest:'.format(i), min_r2_rf)
  #   print('Min MAPE for Random Forest:'.format(i), min_mape_rf)
  #   print()

  #   # Linear Regression
  #   lr = LinearRegression()
  #   lr.fit(X_train, y_train)
  #   lr_predicts = lr.predict(X_test)
  #   # Calculate RMSE
  #   rmse_lr = math.sqrt(mean_squared_error(y_test, lr_predicts))
  #   # Calculate R2 Score
  #   r2_lr = r2_score(y_test, lr_predicts)
  #   # Calculate mean absolute error (MAE NOT MAPE)
  #   # mape_lr = mean_absolute_error(y_test, lr_predicts)
  #   mape_lr = mean_absolute_percentage_error(y_test, rf_predicts)

  #   if rmse_lr < min_rmse_lr:
  #     min_rmse_lr = rmse_lr
  #   if r2_lr < min_r2_lr:
  #     min_r2_lr = r2_lr
  #   if mape_lr < min_mape_lr:
  #     min_mape_lr = mape_lr

  #   # a={'true': y_test, 'predict':predicts}
  #   # display(pd.DataFrame(a))

  #   rmses_lr.append(min_rmse_lr)
  #   r2s_lr.append(min_r2_lr)
  #   mapes_lr.append(min_mape_lr)

  #   print('Min RMSE for Linear Regression:'.format(i), min_rmse_lr)
  #   print('Min R2 for Linear Regression:'.format(i), min_r2_lr)
  #   print('Min MAPE for Linear Regression:'.format(i), min_mape_lr)
  #   print()
  #   print()

  #   i += 1
  # if (idx == 2):
  #     print("Original data")
  #     print()
  # else:
  #     print("Data without " + columns[idx])
  #     print()
  # print()
  # print('Average RMSE on all folds using Random Forest:', np.average(rmses_rf))
  # print('Average R2 on all folds using Random Forest:', np.average(r2s_rf))
  # print('Average MAPE on all folds using Random Forest:', np.average(mapes_rf))

  # print()
  # print('Average RMSE on all folds using Linear Regression:', np.average(rmses_lr))
  # print('Average R2 on all folds using Linear Regression:', np.average(r2s_lr))
  # print('Average MAPE on all folds using Linear Regression:', np.average(mapes_lr))

  # List of RMSE on all folds
# rmses_rf = []
# mapes_rf = []
# r2s_rf = []

# rmses_lr = []
# mapes_lr = []
# r2s_lr = []

  # Fold tracking
  i = 1
  # Depth of Random Forest
  depth = 200
  # Number of split
  split_number = 10

  map_depth_rmse = {}
  map_depth_r2 = {}
  map_depth_mape = {}

  # How many number want to round with loss (1.424 if precision =3 for example)
  precision = 5

  def MAPE(Y_actual,Y_Predicted):
      mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
      return mape

  def get_key(dict, val):
    for key, value in dict.items():
          if val == value:
              return key

    return "key doesn't exist"

  # kf = KFold(n_splits=split_number, random_state=None, shuffle=None)
  # kf.get_n_splits(X)

  # for train_index, test_index in kf.split(X):
  #   min_rmse_rf = 10000
  #   min_r2_rf = 10000
  #   min_mape_rf = 10000

  #   min_rmse_lr = 10000
  #   min_r2_lr = 10000
  #   min_mape_lr = 10000

  #   # print("TRAIN:", train_index, "TEST:", test_index)
  #   X_train, X_test = X_arr[train_index], X_arr[test_index]
  #   y_train, y_test = y_arr[train_index], y_arr[test_index]
  #   # print(len(train_index), len(test_index))

  #   print('Fold {}:'.format(i))

    # Random Forest
  for d in range(100, depth):
    kf = KFold(n_splits=split_number, random_state=None, shuffle=True)
    kf.get_n_splits(X)

    rmses_rf = []
    mapes_rf = []
    r2s_rf = []

    rmses_lr = []
    mapes_lr = []
    r2s_lr = []

    for train_index, test_index in kf.split(X):
      min_rmse_rf = 10000
      min_r2_rf = 0
      min_mape_rf = 10000

      # min_rmse_lr = 10000
      # min_r2_lr = 10000
      # min_mape_lr = 10000

      # print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X_arr[train_index], X_arr[test_index]
      y_train, y_test = y_arr[train_index], y_arr[test_index]
      # print(len(train_index), len(test_index))

      # print('Fold {}:'.format(i))
      rf = RandomForestRegressor(n_estimators = d, random_state = 20)
      rf.fit(X_train, y_train)
      rf_predicts = rf.predict(X_test)
      # Calculate RMSE
      rmse_rf = math.sqrt(mean_squared_error(y_test, rf_predicts))
      # Calculate R2 Score
      r2_rf = r2_score(y_test, rf_predicts)
      # Calculate mean absolute error (MAE NOT MAPE)
      # mape_rf = mean_absolute_error(y_test, rf_predicts)
      mape_rf = MAPE(y_test, rf_predicts)

      # With each depth, calculate and save losses of all fold
      rmses_rf.append(rmse_rf)
      r2s_rf.append(r2_rf)
      mapes_rf.append(mape_rf)

    # After each depth, save average loss to find best depth
    average_rmse_rf = round(np.average(rmses_rf), precision)
    average_r2_rf = round(np.average(r2s_rf), precision)
    average_mape_rf = round(np.average(mape_rf), precision)

    map_depth_rmse[d] = average_rmse_rf
    map_depth_r2[d] = average_r2_rf
    map_depth_mape[d] = average_mape_rf
    
    print('Depth =', d)
    print('Average RMSE on all folds using Random Forest:', average_rmse_rf)
    print('Average R2 on all folds using Random Forest:', average_r2_rf)
    print('Average MAPE on all folds using Random Forest:', average_mape_rf)
    print()

  # print(map_depth_rmse)
  # print(map_depth_r2)

  min_rmse = min(map_depth_rmse.values())
  max_r2 = max(map_depth_r2.values())
  min_mape = min(map_depth_mape.values())

  print('Best depth for RMSE is {} with RMSE = {}'.format(get_key(map_depth_rmse, min_rmse), min_rmse))
  print('Best depth for R2 is {} with R2 = {}'.format(get_key(map_depth_r2, max_r2), max_r2))
  print('Best depth for MAPE is {} with MAPE = {}'.format(get_key(map_depth_mape, min_mape), min_mape))


---------------------------------------------

Data without cluster

---------------------------------------------
Depth = 100
Average RMSE on all folds using Random Forest: 9.66185
Average R2 on all folds using Random Forest: 0.58746
Average MAPE on all folds using Random Forest: 33.22839

Depth = 101
Average RMSE on all folds using Random Forest: 9.70642
Average R2 on all folds using Random Forest: 0.58493
Average MAPE on all folds using Random Forest: 40.50461

Depth = 102
Average RMSE on all folds using Random Forest: 9.65604
Average R2 on all folds using Random Forest: 0.58955
Average MAPE on all folds using Random Forest: 36.65961

Depth = 103
Average RMSE on all folds using Random Forest: 9.69505
Average R2 on all folds using Random Forest: 0.5842
Average MAPE on all folds using Random Forest: 37.11576

Depth = 104
Average RMSE on all folds using Random Forest: 9.69656
Average R2 on all folds using Random Forest: 0.58569
Average MAPE on all folds using Random Forest: 32.19078

D